# DS 6011: Read in data

- Andrew J. Graves, Cory Clayton, Joon Soh, Gabe Yohe
- 10/09/20

In [25]:
# Get data from internet using wget
def wget_raw_edf(f):
    
    from wget import download
    from os import remove
    from mne.io import read_raw_edf
    
    wg = download(f) # download the data locally (temporarily)
    raw = read_raw_edf(wg) # read data as an MNE object
    remove(wg) # delete the file locally
    return raw

# Specifies which files to iterate through
def gen_iter(param, n_params):
    
    if param != 'all': # for subset of the data
        param_iter = param
    else: # for all of the data
        param_iter = range(1, n_params+1)
    return param_iter

# Need to build a dataset class!
def motormovement_imagine():
    
    git_repo = 'https://github.com/VIXSoh/SRM/raw/master/data/' # data repository
    subj_count = 109 # total number of subjects
    task_count = 14 # total number of tasks
    return git_repo, subj_count, task_count

def read_data(dataset, subjects='all', tasks='all'):
    
    # Dataset attributes (make a dataset class later)
    dir_link, n_subj, n_task = dataset
    
    missing = [] # initialize missing file list
    subj_iter = gen_iter(subjects, n_subj) # get subject iterator
    task_iter = gen_iter(tasks, n_task) # get task iterator

    # Iterate through subjects
    for subj in subj_iter:
    
        # Build this in the dataset class later!
        subj_num = f"S{str(subj).rjust(3, '0')}" # Subject number string
    
        # Iterate through tasks
        for task in task_iter:
            # Build this in the dataset class later!
            task_num = f"R{str(task).rjust(2, '0')}" # Task number string
            try:
                # Build this in the dataset class later
                file_path = f'{dir_link}{subj_num}/{subj_num}{task_num}.edf' # Get the file path
                wget_raw_edf(file_path)
                # Send to preproc function next
            except:
                print(f'---\nThis file does not exist: {file_path}\n---')
                missing.append(file_path)

In [26]:
read_data(motormovement_imagine())

100% [..........................................................................] 1275936 / 1275936Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S001R01 (1).edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
100% [..........................................................................] 1275936 / 1275936Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S001R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
100% [..........................................................................] 2596896 / 2596896Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S001R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
100% [.................................

<ipython-input-25-8a1596ccd5ad>:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_edf(wg) # Read data as an MNE object


100% [..........................................................................] 2047872 / 2047872Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S100R04.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-25-8a1596ccd5ad>:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_edf(wg) # Read data as an MNE object


100% [..........................................................................] 2047872 / 2047872Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S100R05.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


...

100% [..........................................................................] 2047872 / 2047872Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S100R13.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-25-8a1596ccd5ad>:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_edf(wg) # Read data as an MNE object


100% [..........................................................................] 2047872 / 2047872Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S100R14.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...


<ipython-input-25-8a1596ccd5ad>:9: RuntimeWarning: Limited 1 annotation(s) that were expanding outside the data range.
  raw = read_raw_edf(wg) # Read data as an MNE object


100% [..........................................................................] 1275936 / 1275936Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S101R01.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
100% [..........................................................................] 1275936 / 1275936Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S101R02.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
100% [..........................................................................] 2596896 / 2596896Extracting EDF parameters from C:\Users\Andrew Graves\Documents\University of Virginia\Course Work\Fall 2020\ds_6011\code\S101R03.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
100% [.....................................

## Manipulates an example subject (dated)

In [ ]:
%matplotlib notebook

# Import modules
import numpy as np
import mne

# Example file from example subject
raw = mne.io.read_raw_edf('S001R04.edf')

# Print the information
raw.info['nchan']

In [ ]:
# Print the EEG data shape
shape = raw.get_data().shape
print(shape)

In [ ]:
# Sampling and Nyquist frequency
samp_freq = raw.info['sfreq']
nyq = samp_freq / 2
print(nyq)

In [ ]:
# Task time
shape[1] / (samp_freq)

In [ ]:
# T0 is baseline, T1 is left fist / both, T2 if right fist / both
ev = mne.events_from_annotations(raw)
print(ev[1])
np.delete(ev[0], 1, axis=1)

In [ ]:
unfilt = raw.plot()

In [ ]:
filt = raw.plot(highpass=nyq-1, lowpass=.5)